# Project 1 Association Analysis

## Task

* Dataset1: Select from kaggle.com / UCI
* Dataset2: Use IBM Quest Synthetic Data Generator 
    * https://sourceforge.net/projects/ibmquestdatagen/ 
    * Generate different datasets
* Implement Apriori Algorithm and apply on these datasets 
    * Hash? Tree? (optional)
    * FP-growth
* Use association analysis tools (e.g. WEKA) to generate association rules from the datasets you generate
* Compare your results

In [1]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt

### Parse Data Generated by IBM Quest Synthetic Data Generator

In [2]:
inputfile = open('data/data_1_3_20_10_5.data', 'r')
outputfile = open('data/data_1_3_20_10_5.csv', 'w')

In [3]:
outputfile.write('CustID,TransID,Item\n')

20

In [4]:
while True:

    s = inputfile.readline().rstrip('\n')
    
    # if this is the eof
    if len(s) == 0:
        break
        
    # Extract dimensions from first line. Cast values to integers from strings.
    CustID, TransID, Item = (int(val) for val in s.split())
    output_str = '%d,%d,%d\n' % (CustID, TransID, Item)
    outputfile.write(output_str)

In [5]:
inputfile.close()
outputfile.close()

### Before Getting Started

In [6]:
# Load data
FILE_PATH = 'data/fcam.csv'
# FILE_PATH = 'data/data_1_3_20_10_5.csv'
df = pd.read_csv(FILE_PATH)
df = df.astype({'Item': str})
# Parameters
minsup = 3
minconf = 0.8
# Candidate 1-itemset
C1_df = df['Item'].value_counts()
# Frequent 1-itemset
L1_df = C1_df.loc[C1_df.values >= minsup]
L1 = L1_df.index.values.tolist()

In [7]:
print(L1)

['f', 'c', 'm', 'b', 'a', 'p']


In [8]:
# Init dictionary for every transaction
trans_num = df['TransID'].max()
transaction_db = {}
for i in range(1, trans_num + 1):
    transaction_db[i] = []
# Extract info from df to dictionary
df_num = len(df)
for i in range(df_num):
    index = df.iloc[i][0]
    item = df.iloc[i][2]    
    transaction_db[index] += [item]

In [9]:
print(transaction_db)

{1: ['a', 'c', 'd', 'f', 'g', 'i', 'm', 'p'], 2: ['a', 'b', 'c', 'f', 'i', 'm', 'o'], 3: ['b', 'f', 'h', 'j', 'o'], 4: ['b', 'c', 'k', 's', 'p'], 5: ['a', 'c', 'e', 'f', 'l', 'm', 'n', 'p']}


### Apriori Algorithm

In [10]:
def Apriori_gen(x, k):
    Ck = []
    for subset in itertools.combinations(x, k):
        # print(subset)
        tmp = [set(item) for item in subset]
        union_result = set()
        for subset2 in itertools.combinations(tmp, k - 1):
            # print(subset2)
            # print(subset2[0].intersection(subset2[1]))
            result = subset2[0]
            for i in range(k - 1):
                result = result.intersection(subset2[i])
            # print(result)
            union_result = union_result.union(result)
        if len(union_result) == k:
            Ck.append(list(union_result))
    return Ck

In [11]:
def Apriori(tdb, L1, minsup):
    Lk = L1
    k = 2
    FreqPat = []
    while Lk != []:
        FreqPat.append(Lk)
        # Use previous Lk to generate Ck
        Ck = Apriori_gen(Lk, k)
        # Count the number of every item in Ck appears in DB
        Ck_freq = {}
        for item in Ck:
            count = 0
            for transaction in tdb.values():
                if all(x in transaction for x in item):
                    count += 1
            Ck_freq[tuple(item)] = count
        # Generate Lk
        Lk = []
        for (key, value) in Ck_freq.items():
            if value >= minsup:
                Lk.append(list(key))
        k += 1
    return FreqPat

In [12]:
Apriori(transaction_db, L1, minsup)

[['f', 'c', 'm', 'b', 'a', 'p'],
 [['c', 'f'],
  ['m', 'f'],
  ['a', 'f'],
  ['c', 'm'],
  ['c', 'a'],
  ['p', 'c'],
  ['m', 'a']],
 [['c', 'm', 'f'], ['c', 'a', 'f'], ['m', 'a', 'f'], ['c', 'a', 'm']],
 [['c', 'a', 'm', 'f']]]

In [13]:
# all(x in ['a', 'c', 'd', 'f', 'g', 'i', 'm', 'p'] for x in ['f', 'c'])

In [14]:
# for item in y:
#     print(item)
#     for transaction in di.values():
#         if all(x in transaction for x in item):
#             print('yes')

In [15]:
# # x = [['B', 'C'], ['A', 'C'], ['B', 'E'], ['C', 'E']]
# # x = [['A', 'B', 'C'], ['A', 'B', 'D'], ['A', 'C', 'D'], ['B', 'C', 'D'], ['A', 'D', 'E']]
# x = [['A', 'B', 'C', 'D'], ['A', 'B', 'C', 'E'], ['A', 'B', 'D', 'E'], 
#      ['A', 'C', 'D', 'E'], ['B', 'C', 'D', 'E'], ['A', 'D', 'E', 'F'], ['A', 'F', 'D', 'E'],]
# k = 5
# L = []
# for subset in itertools.combinations(x, k):
#     print(subset)
#     tmp = [set(item) for item in subset]
#     union_result = set()
#     for subset2 in itertools.combinations(tmp, k - 1):
#         # print(subset2)
#         # print(subset2[0].intersection(subset2[1]))
#         result = subset2[0]
#         for i in range(k - 1):
#             result = result.intersection(subset2[i])
#         # print(result)
#         union_result = union_result.union(result)
#     if len(union_result) == k:
#         L.append(list(union_result))
# print(L)        

### FP-growth

In [16]:
class HeaderTableNode:
    def __init__(self):
        self.head = None
        self.tail = None

class FPtreeNode:
    def __init__(self, val, parent=None):
        self.val = val
        self.count = 1
        self.parent = parent
        self.children = []
        self.next = None
    def insert_frequent_items(self, items, hdtable):
        # If there is no frequent item
        if len(items) == 0:
            return
        item = items[0]
        for child in self.children:
            if child.val == item:
                child.count += 1
                child.insert_frequent_items(items[1:], hdtable)
                return
        # If cannot find the item among children
        new_child = FPtreeNode(item, self)
        # Add new node to header table
        if hdtable[item].head == None:
            hdtable[item].head = new_child
            hdtable[item].tail = new_child
        else:
            hdtable[item].tail.next = new_child
            hdtable[item].tail = new_child
        # Add new node to current node's children
        self.children.append(new_child)
        new_child.insert_frequent_items(items[1:], hdtable)

class CondPatternBase:
    def __init__(self, pattern, freq):
        self.pattern = pattern
        self.freq = freq

class FreqPattern:
    def __init__(self, pattern, support):
        self.pattern = pattern
        self.support = support

In [17]:
# Init dictionary for ordered frequent items of every transaction
ofi = {}
for i in range(1, trans_num + 1):
    ofi[i] = []
# Construct ordered frequent items of every transaction
for i in range(1, trans_num + 1):
    for item in L1:
        if item in transaction_db[i]:
            ofi[i] += [item]

# Init header table
HeaderTable = {}
for item in L1:
    new_node = HeaderTableNode()
    HeaderTable[item] = new_node

# Construct FP-tree
FPtree = FPtreeNode('root')
for i in range(1, trans_num + 1):
    FPtree.insert_frequent_items(ofi[i], HeaderTable)



In [18]:
ofi

{1: ['f', 'c', 'm', 'a', 'p'],
 2: ['f', 'c', 'm', 'b', 'a'],
 3: ['f', 'b'],
 4: ['c', 'b', 'p'],
 5: ['f', 'c', 'm', 'a', 'p']}

In [19]:
# Generate conditional pattern base
CondBase = {}
for item in L1:
    # Init
    CondBase[item] = []
    # Start from head, and no need to traverse the leaf node
    listnode = HeaderTable[item].head
    treenode = listnode.parent
    # Traversal of linked-list
    while True:
        pattern = []
        # Traversal of tree
        while True:
            if treenode.val == 'root':
                # print()
                break
            # print('%s ' % treenode.val, end = '')
            pattern.insert(0, treenode.val)
            treenode = treenode.parent
        # Create a new base for this item
        if len(pattern) > 0:
            new_base = CondPatternBase(pattern, listnode.count)
            CondBase[item].append(new_base)
            # print('item = %s, count = %d: ' % (item, listnode.count), end = '\t')
            # print(pattern)
        # Reach the end of the list of this item
        if listnode.next == None:
            break
        # Continue to next node in the list, and no need to traverse the leaf node
        listnode = listnode.next
        treenode = listnode.parent



In [20]:
CondBase

{'f': [],
 'c': [<__main__.CondPatternBase at 0x111c3b210>],
 'm': [<__main__.CondPatternBase at 0x11177c390>],
 'b': [<__main__.CondPatternBase at 0x111c3b290>,
 'a': [<__main__.CondPatternBase at 0x111c3b350>,
 'p': [<__main__.CondPatternBase at 0x111c3b3d0>,
  <__main__.CondPatternBase at 0x111c3b410>]}

In [21]:
# Accumulate the count for each item in the base
freq = {}
for item1 in L1:
    freq[item1] = {}
    for item2 in L1:
        freq[item1][item2] = 0
    for base in CondBase[item1]:
        for item3 in base.pattern:
            freq[item1][item3] += base.freq    
    # print(item1)
    print(freq[item1])



{'f': 0, 'c': 0, 'm': 0, 'b': 0, 'a': 0, 'p': 0}
{'f': 3, 'c': 0, 'm': 0, 'b': 0, 'a': 0, 'p': 0}
{'f': 3, 'c': 3, 'm': 0, 'b': 0, 'a': 0, 'p': 0}
{'f': 2, 'c': 2, 'm': 1, 'b': 0, 'a': 0, 'p': 0}
{'f': 3, 'c': 3, 'm': 3, 'b': 1, 'a': 0, 'p': 0}
{'f': 2, 'c': 3, 'm': 2, 'b': 1, 'a': 2, 'p': 0}


In [22]:
# (conditional FP-tree)
condFPtree = {}
for item1 in L1:
    tmp_pattern = []
    for item2 in L1:
        if freq[item1][item2] >= minsup:
            tmp_pattern.append(item2)
    if len(tmp_pattern) > 0:
        condFPtree[item1] = tmp_pattern

In [23]:
print(condFPtree)

{'c': ['f'], 'm': ['f', 'c'], 'a': ['f', 'c', 'm'], 'p': ['c']}


In [24]:
# Generate frequent patterns
FreqPat = []
for key in condFPtree:
    x = condFPtree[key]
    for L in range(1, len(x)+1):
        for subset in itertools.combinations(x, L):
            pat = list(subset)
            pat.append(key)
            FreqPat.append(pat)
print(FreqPat)



[['f', 'c'], ['f', 'm'], ['c', 'm'], ['f', 'c', 'm'], ['f', 'a'], ['c', 'a'], ['m', 'a'], ['f', 'c', 'a'], ['f', 'm', 'a'], ['c', 'm', 'a'], ['f', 'c', 'm', 'a'], ['c', 'p']]


In [25]:
# Rule generation
# for each frequent itemset m do 
#     for each subset p of m do
#         if (support(m)/support(p)) >= min_confidence then 
#             output the rule p->(m-p)

### Generate csv for WEKA

In [26]:
df = pd.read_csv('data/data_1_3_20_10_5.csv')

In [27]:
# Init dictionary for every transaction
trans_num = df['TransID'].max()
di = {}
for i in range(1, trans_num + 1):
    di[i] = []
# Extract info from df to dictionary
df_num = len(df)
for i in range(df_num):
    index = df.iloc[i][0]
    item = df.iloc[i][2]    
    di[index] += [item]

In [28]:
outputfile = open('data/weka_data_1_3_20_10_5.csv', 'w')

In [29]:
outputfile.write('TransID')

title = list(df['Item'].unique())
for item in title:
    outputfile.write(',%s' % item)
outputfile.write('\n')

1

In [30]:
trans_num = df['TransID'].max()

In [31]:
for i in range(1, trans_num + 1):
    outputfile.write('%d' % i)
    for item in title:
        if item in di[i]:
            outputfile.write(',1')
        else:
            outputfile.write(',0')
    outputfile.write('\n')

In [32]:
outputfile.close()

In [33]:
df_weka = pd.read_csv('data/weka_data_1_3_20_10_5.csv')
df_weka.head()

,TransID,4089,8704,9205,9430,12679,12779,18927,19970,38,...,17679,15854,16381,9914,7711,14049,19119,17637,5085,11480
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Find and answer

What are rules with
* High support, high confidence ? 
* High support, low confidence ? 
* Low support, low confidence ? 
* Low support, high confidence ?